# Model testing

    a. data layout (select channels, with/without raw signals, etc.)
    b. model selection (with various hyperparameters)
    c. feature learning / dimension reduction

In [1]:
# Module imports
import warnings

warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pickle
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.metrics import accuracy_score
from scipy.stats import skew, kurtosis
from sklearn.model_selection import learning_curve
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# Global parameters
CV = 5
SEED = 24
STRATIFY = True
TEST_SIZE = 0.2
ROOT = Path('/tmp/working/data/processed')  # use the current directory as the root

In [3]:
def select_channels(data_df, channels):
    """
    select a collection of channels, and stack signals of all channels horizontally 
    so each row represent to one participant
    """
    return data_df.loc[(slice(None), slice(None), channels), :].unstack('channel')

def prep_data(dataframe):
    """
    Prepare features X and labels y from given dataset
    In the dataframe the index 'healthy' is the label
    """
    X = dataframe.sample(frac=1) #  sample(frac=1) method will shuffle the data
    y = dataframe.reset_index('healthy')['healthy']  # index 'healthy' is used as the class label
    y = y ^ 1
    return X, y

def draw_learning_curve(X, y, model, steps=5, cv=3):
    """
    Function to draw learning curve
    """
    max_size = X.shape[0] // 3 * 2
    train_sizes = np.linspace(10, max_size, steps, dtype=np.int).tolist()
    _, train_scores, validation_scores = learning_curve(model, X, y, train_sizes=train_sizes, cv=cv)
    plt.plot(train_sizes, train_scores.mean(axis=1), label='train score')
    plt.plot(train_sizes, validation_scores.mean(axis=1), label='validation score')
    plt.title('Learning curve')
    plt.legend()
    plt.show()

def run_models(X, y, dim_reductions, classifiers, n_splits=3, test_size=0.2, stratify=True, seed=None):
    """
    Function to run various combinations of dimention reduction and model
    """
    combinations = [[dim_red, model] for dim_red in dim_reductions for model in classifiers]
    
    # choose the dataset to work with

    # Training testing data spliting
    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        y, 
                                                        test_size=test_size, 
                                                        stratify=y if stratify else None)
    print('Training data dimension: samples {}; features {}'.format(*X_train.shape))
    print('Testing data dimension: samples {}; features {}'.format(*X_test.shape))
    
    # memorize the cross-validation spliting
    if stratify:
        cv = list(StratifiedKFold(n_splits=n_splits, random_state=seed).split(X_train, y_train))
    else:
        cv = list(KFold(n_splits=n_splits, random_state=seed).split(X_train))
        
    # run the combinations
    for steps in combinations:
        p = Pipeline(steps)
        try:
            # try to use multiple cpu cores
            cv_result = cross_val_score(p, X_train, y_train, cv=cv, n_jobs=1)
        except OSError:
            cv_result = cross_val_score(p, X_train, y_train, cv=cv, n_jobs=1)
        cv_score = cv_result.mean()
        cv_std = cv_result.std()
        step_names = [name for name, _ in steps]
        print('Model ({}) CV score: {:.3f} +- {:.3f}'.format(
            '-'.join(step_names), 
            cv_score,
            cv_std
        ))
        cls = p.fit(X_train, y_train)
        y_pred = cls.predict(X_test)
        test_score = accuracy_score(y_test, y_pred)
        print('Model ({}) test score: {:.3f}'.format(
            '-'.join(step_names), 
            test_score))
        print()

### features only

In [4]:
# Load data
data_file = ROOT / 'lpp_step_2_unioned_feature.pkl'
data_df = pickle.load(data_file.open('rb'))
data_df.head()

positive                                          \
                          max       min     range       std       avg   
healthy id   channel                                                    
0       2000 CP1      4.01125  -8.76025  12.77150  2.014002  0.211150   
             CP2      6.57300  -7.31650  13.88950  2.537050  2.007386   
             Cz       3.63265 -10.40910  14.04175  2.555168 -0.748934   
             FC1      2.26965 -11.08525  13.35490  2.536261 -2.241016   
             FC2      2.94560 -11.05345  13.99905  2.570983 -1.327018   

                                                        neutral            \
                          skew  kurtosis signaltonoise      max       min   
healthy id   channel                                                        
0       2000 CP1     -1.568806  4.167685      0.104841  4.01125  -8.76025   
             CP2     -1.057286  1.371455      0.791228  6.57300  -7.31650   
             Cz      -1.210035  1.981335     -0.293105  3.63265 -10.40910   
             FC1     -1.024286  0.799563     -0.883591  2.26965 -11.08525   
             FC2     -1.052346  1.406583     -0.516152  2.94560 -11.05345   

                      ...                         lpp diff                  \
                      ...  kurtosis signaltonoise      max  min range  std   
healthy id   channel  ...                                                    
0       2000 CP1      ...  4.167685      0.104841      0.0  0.0   0.0  0.0   
             CP2      ...  1.371455      0.791228      0.0  0.0   0.0  0.0   
             Cz       ...  1.981335     -0.293105      0.0  0.0   0.0  0.0   
             FC1      ...  0.799563     -0.883591      0.0  0.0   0.0  0.0   
             FC2      ...  1.406583     -0.516152      0.0  0.0   0.0  0.0   

                                                       
                      avg skew kurtosis signaltonoise  
healthy id   channel                                   
0       2000 CP1      0.0  0.0     -3.0           NaN  
             CP2      0.0  0.0     -3.0           NaN  
             Cz       0.0  0.0     -3.0           NaN  
             FC1      0.0  0.0     -3.0           NaN  
             FC2      0.0  0.0     -3.0           NaN  

[5 rows x 24 columns]

In [5]:
data_df.dropna(axis=1, inplace=True)

In [6]:
# model configurations
models = [
    ('svm linear', SVC(kernel='linear', C=0.025)),
    ('svm rbf', SVC(C=0.025)),
    ('random forest', RandomForestClassifier(n_estimators=100, random_state=SEED)),
    ('Ada boost', AdaBoostClassifier(learning_rate=0.75, random_state=SEED)),
    ('Extra tree', ExtraTreesClassifier(n_estimators=100, random_state=SEED)),
    ('Gradient boosting trees', GradientBoostingClassifier(n_estimators=100, random_state=SEED)),
#     ('xgb', XGBClassifier(random_state=SEED))
]

# dimension reduction configurations
dim_reductions = [
    ('none', None),
#     ('pca 100', PCA(n_components=100)),
#     ('lle', LocallyLinearEmbedding(n_components=50))
]

In [7]:
channels = ['Cz']
X, y = prep_data(
    select_channels(
        data_df,
        channels
    )
)

run_models(
    X,
    y,
    dim_reductions=dim_reductions,
    classifiers=models,
    n_splits=CV,
    test_size=TEST_SIZE,
    stratify=True,
    seed=SEED
)

Training data dimension: samples 16; features 23
Testing data dimension: samples 4; features 23
Model (none-svm linear) CV score: 0.250 +- 0.224
Model (none-svm linear) test score: 0.500

Model (none-svm rbf) CV score: 0.550 +- 0.245
Model (none-svm rbf) test score: 0.500

Model (none-random forest) CV score: 0.700 +- 0.245
Model (none-random forest) test score: 0.750

Model (none-Ada boost) CV score: 0.800 +- 0.187
Model (none-Ada boost) test score: 0.750

Model (none-Extra tree) CV score: 0.550 +- 0.245
Model (none-Extra tree) test score: 0.750

Model (none-Gradient boosting trees) CV score: 0.650 +- 0.200
Model (none-Gradient boosting trees) test score: 0.750



### raw signals plus features

In [8]:
# Load data
data_file1 = ROOT / 'lpp_step_2_unioned.pkl'
data_df = pickle.load(data_file1.open('rb'))
data_df.head()
data_file2 = ROOT / 'lpp_step_2_unioned_feature.pkl'
feature_df = pickle.load(data_file2.open('rb'))
combined_df = pd.concat([data_df, feature_df], keys=['raw', 'feature'], axis=1)
combined_df.head()

raw                                          \
                      positive                                           
                             0         1         2         3         4   
healthy id   channel                                                     
0       2000 CP1      0.265516  0.348982  0.436767  0.526092  0.614672   
             CP2     -0.220014 -0.143054 -0.065878  0.008401  0.077478   
             Cz       0.622771  0.720553  0.824127  0.929619  1.033507   
             FC1      1.090608  1.169859  1.252184  1.334274  1.413130   
             FC2      1.016447  1.073468  1.133076  1.192742  1.250540   

                                                                        ...  \
                                                                        ...   
                             5         6         7         8         9  ...   
healthy id   channel                                                    ...   
0       2000 CP1      0.701017  0.784011  0.862412  0.934582  0.998311  ...   
             CP2      0.139735  0.193951  0.238885  0.273059  0.294876  ...   
             Cz       1.132992  1.225901  1.310299  1.384012  1.444595  ...   
             FC1      1.486880  1.554500  1.615258  1.668250  1.712271  ...   
             FC2      1.305344  1.356434  1.402875  1.443249  1.475707  ...   

                       feature                                              \
                       neutral               lpp diff                        
                      kurtosis signaltonoise      max  min range  std  avg   
healthy id   channel                                                         
0       2000 CP1      4.167685      0.104841      0.0  0.0   0.0  0.0  0.0   
             CP2      1.371455      0.791228      0.0  0.0   0.0  0.0  0.0   
             Cz       1.981335     -0.293105      0.0  0.0   0.0  0.0  0.0   
             FC1      0.799563     -0.883591      0.0  0.0   0.0  0.0  0.0   
             FC2      1.406583     -0.516152      0.0  0.0   0.0  0.0  0.0   

                                                  
                                                  
                     skew kurtosis signaltonoise  
healthy id   channel                              
0       2000 CP1      0.0     -3.0           NaN  
             CP2      0.0     -3.0           NaN  
             Cz       0.0     -3.0           NaN  
             FC1      0.0     -3.0           NaN  
             FC2      0.0     -3.0           NaN  

[5 rows x 4224 columns]

In [12]:
combined_df.dropna(axis=1, inplace=True)

In [15]:
# model configurations
models = [
    ('svm linear', SVC(kernel='linear', C=0.025)),
    ('svm rbf', SVC(C=0.025)),
    ('random forest', RandomForestClassifier(n_estimators=100, random_state=SEED)),
    ('Ada boost', AdaBoostClassifier(learning_rate=0.75, random_state=SEED)),
    ('Extra tree', ExtraTreesClassifier(n_estimators=100, random_state=SEED)),
    ('Gradient boosting trees', GradientBoostingClassifier(n_estimators=100, random_state=SEED)),
#     ('xgb', XGBClassifier(random_state=SEED))
    
]

# dimension reduction configurations
dim_reductions = [
    ('none', None),
    ('pca 100', PCA(n_components=10)),
    ('lle', LocallyLinearEmbedding(n_components=10))
]

In [16]:
channels = ['Cz']
X, y = prep_data(
    select_channels(
        combined_df,
        channels
    )
)

run_models(
    X,
    y,
    dim_reductions=dim_reductions,
    classifiers=models,
    n_splits=CV,
    test_size=TEST_SIZE,
    stratify=True,
    seed=SEED
)

Training data dimension: samples 16; features 4223
Testing data dimension: samples 4; features 4223
Model (none-svm linear) CV score: 0.650 +- 0.200
Model (none-svm linear) test score: 0.250

Model (none-svm rbf) CV score: 0.650 +- 0.200
Model (none-svm rbf) test score: 0.000

Model (none-random forest) CV score: 0.600 +- 0.122
Model (none-random forest) test score: 0.250

Model (none-Ada boost) CV score: 0.700 +- 0.245
Model (none-Ada boost) test score: 0.500

Model (none-Extra tree) CV score: 0.600 +- 0.122
Model (none-Extra tree) test score: 0.250

Model (none-Gradient boosting trees) CV score: 0.550 +- 0.100
Model (none-Gradient boosting trees) test score: 0.500

Model (pca 100-svm linear) CV score: 0.650 +- 0.200
Model (pca 100-svm linear) test score: 0.000

Model (pca 100-svm rbf) CV score: 0.550 +- 0.100
Model (pca 100-svm rbf) test score: 0.500

Model (pca 100-random forest) CV score: 0.650 +- 0.200
Model (pca 100-random forest) test score: 0.500

Model (pca 100-Ada boost) CV s